# This notebooks opens inTRACKtive directly from the notebook

Dependencies

In [1]:
import pandas as pd
from pathlib import Path
from intracktive.convert import dataframe_to_browser

Load tracks data for C. elegans (or replace by your own data)

In [ ]:
df = pd.read_csv("https://public.czbiohub.org/royerlab/zoo/C_elegans/tracks.csv")
df

Open intracktive directly from the data

In [3]:
# Choose the location where the Zarr file will be stored, providing an empty path (Path()) means that the data will be stored in a temporary location
zarr_dir = Path()
# zarr_dir = '/path/on/your/computer/

# dataframe_to_browser(df, zarr_dir)

The `dataframe_to_browser` function sequentially:
1. converts pd.dataFrame to Zarr
2. saves the Zarr in the specified location
3. spins up a localhost at that location
4. launches a browser window of inTRACKtive with as dataUrl the zarr in the localhost, all in a single click

---------------------------------------------------------------

[OPTIONAL] add attributes to color the cells (takes ~20 sec to run)

In [ ]:
# Add a new column 'lineage' to the dataframe (representing the lineage of each track)
df2 = df.copy()
df2.sort_values(by=["track_id", "t"], inplace=True)
df2['lineage'] = -1
df2.loc[df2['track_id'] <= 1, 'lineage'] = df2['track_id']      # Set lineage for cells in first frame
for idx, row in df2.iterrows():                                 # Looop through all rows and find lineage for each cell
    if row['lineage'] == -1:
        parent_row = df2[df2['track_id'] == row['parent_track_id']] 
        if not parent_row.empty:
            df2.at[idx, 'lineage'] = parent_row.iloc[0]['lineage']


# Open the data in inTRACKtive
dataframe_to_browser(df2, zarr_dir, extra_cols = ['lineage'])